In [21]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd


# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


# Estandarización variables numéricas y Codificación variables categóricas
# ------------------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler

# Gestión datos desbalanceados
# ------------------------------------------------------------------------------
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek

# Para separar los datos en train y test
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split

#  Gestión de warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [22]:
df = pd.read_pickle('../datos/setas/setas_limpio.pkl')

In [23]:
df.isnull().sum()

class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-color              0
stem-height             0
stem-width              0
stem-color              0
has-ring                0
ring-type               0
habitat                 0
season                  0
dtype: int64

## Estandarizar las variables numéricas de vuestro set de datos

In [24]:
# nos hacemos una copia del df que acabamos de cargar

df = df.copy()

In [25]:
# iniciamos el método para escalar

scaler = StandardScaler()

In [26]:
numericas = df.select_dtypes(include = np.number)
numericas.head()

,cap-diameter,stem-height,stem-width
353,6.87,6.88,13.64
354,8.59,9.15,17.34
355,5.95,7.54,12.73
356,6.51,6.80,12.92
357,7.66,8.55,14.98


In [27]:
# ahora ya podemos ajustar nuestros datos.  

scaler.fit(numericas)

# transformamos los datos

X_escaladas = scaler.transform(numericas)

# por último convertiremos el array que nos devuelve en un dataframe. 

numericas_estandar = pd.DataFrame(X_escaladas, columns = numericas.columns)
numericas_estandar.head(2)

,cap-diameter,stem-height,stem-width
0,0.291853,0.358694,0.400040
1,0.811983,1.407073,0.910123


In [28]:
# lo unimos al dataframe original 

df = pd.concat([df, numericas_estandar], axis = 1)

# chequeamos que esta todo bien
df.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season,cap-diameter,stem-height,stem-width
353,p,6.87,x,g,n,f,e,w,6.88,13.64,w,t,p,d,a,1.788738,2.436978,0.762613
354,p,8.59,p,g,n,f,e,w,9.15,17.34,w,t,p,d,a,1.831074,2.238386,0.471728
355,p,5.95,p,g,n,f,e,w,7.54,12.73,w,t,p,d,u,1.380497,2.173728,0.656460
356,p,6.51,x,g,n,f,e,w,6.80,12.92,w,t,p,d,a,1.825026,2.085979,0.259423
357,p,7.66,x,g,n,f,e,w,8.55,14.98,w,t,p,d,a,2.190931,2.400030,0.759856


## Codificar las variables categóricas

Nuestras variables categóricas son: 'class', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-color', 'stem-color',
       'has-ring', 'ring-type', 'habitat', 'season'

In [29]:
categoricas = df.select_dtypes('category')
categoricas.columns

Index(['class', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-color', 'stem-color',
       'has-ring', 'ring-type', 'habitat', 'season'],
      dtype='object')

#### Variables nominales

In [ ]:
lista_columnas = ["embarked", "maturity", "adult_male", "alone"]

df_encoded = pd.DataFrame()


for columna in lista_columnas:
    df_dummies = pd.get_dummies(df[columna], prefix_sep = "_", prefix = columna, dtype = int)

    df_encoded = pd.concat([df_encoded, df_dummies], axis = 1)

In [ ]:
# el siguiente paso sería unir este dataframe con nuestro dataframe original para tener todos los datos en un mismo df

df_final = pd.concat([df, df_encoded], axis = 1)
df_final.head()

In [ ]:
# por último nos queda eliminar las columnas categóricas originales porque ya no nos hacen falta. 

df_final.drop(lista_columnas, axis = 1, inplace=True)
df_final.head(2)

#### Variables ordinales

In [ ]:
# definimos el diccionario

map_sex = {"male": 0, "female": 1}

In [ ]:
df_final["sex"] = df_final["sex"].map(map_sex)
df_final.head(2)

## Chequear si vuestros datos están balanceados. 

In [ ]:
# graficamos nuestra variable respuesta

plt.figure(figsize=(8,5)) # para cambiar el tamaño de la figura

fig1 = sns.countplot(data = df, x = "class", color = "mediumaquamarine", edgecolor='black')
fig1.set(xticklabels=["Edible", "Poisonous"]) 
plt.show(); 

In [36]:
# para aplicar este método lo primero que tenemos que hacer es separar en X e y y en train y test como aprendimos en la lecciones de regresion lineal
y = df['class']
X = df.drop('class', axis=1)


#dividimos en sets de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [38]:
# iniciamos el método
os_us = SMOTETomek()

# ajustamos el modelo
X_train_res, y_train_res = os_us.fit_resample(X_train, y_train) 

TypeError: '<' not supported between instances of 'str' and 'float'

In [ ]:
y_train.value_counts()

In [ ]:
y_train_res.value_counts()

In [ ]:
# comprobemos como han quedado ahora las categorías después del ajuste

print (f"Distribución antes del ajuste \n {y_train.value_counts()}" )
print("..............................................................")
print (f"Distribución después del ajuste \n {y_train_res.value_counts()}")

## Guardad el dataframe 

In [ ]:
df.to_pickle('../datos/setas/setas_encod.pkl')